<a href="https://colab.research.google.com/github/Forbusinessuseyukikoishiguro/---PDF-20250403/blob/main/_%E2%91%A2%E3%80%90note%E5%85%AC%E9%96%8B%E3%80%91CSV%E3%83%95%E3%82%A1%E3%82%A4%E3%83%AB%E5%87%BA%E5%8A%9B%E5%AE%9F%E8%A3%85%E3%81%97%E3%81%9F%E3%80%8C%E5%B0%8F%E8%AA%AC%E5%AE%B6%E3%81%AB%E3%81%AA%E3%82%8D%E3%81%86%E3%80%8D%E3%83%87%E3%83%BC%E3%82%BF%E6%8A%BD%E5%87%BA%E6%A4%9C%E7%B4%A2%E3%83%84%E3%83%BC%E3%83%AB20250412.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#③「小説家になろう」データ抽出検索ツール
主な新機能

タブ付きインターフェース

「基本検索」タブ - シンプルな検索条件
「詳細検索」タブ - より細かいフィルター条件


詳細な検索条件

文字数範囲、会話率、挿絵数
読了時間範囲
詳細ジャンル（例：異世界恋愛、現実世界恋愛）
作品要素（異世界転生/転移、残酷描写あり、BL/GL、R15）
連載状況（長期連載停止中を除外など）


表示機能の強化

文字数とブックマーク数による背景色のグラデーション表示
あらすじの適切な長さへの切り詰め
検索結果件数の表示


CSVエクスポート機能

検索結果をCSVファイルとして保存可能
ファイル名に検索キーワードと日時を含める
Google Colabでのダウンロード対応


使い方ガイド

検索条件の説明
検索のコツ
検索結果の見方



使い方

Google Colabで新しいノートブックを開きます
コードセルにこのスクリプトを貼り付けて実行します
使い方ガイドが表示され、その後に検索UIが表示されます
「基本検索」タブと「詳細検索」タブで条件を設定し、「検索」ボタンをクリックします
検索結果が表示されたら、「CSVエクスポート」ボタンでデータを保存できます

検索のコツ

大ジャンルと詳細ジャンル両方の絞り込みが可能
文字数と会話率の組み合わせで読みやすい作品を見つけやすい
長編作品を探す場合は「min_length=50000」などと指定
「連載停止中を除く」設定で更新が途絶えていない作品だけを表示
R18作品を検索する場合は年齢確認が必要

このツールを使って、あなた好みの「小説家になろう」作品を効率的に見つけてください！

In [4]:
import requests
import pandas as pd
from IPython.display import display, HTML
import time
import urllib.parse

def search_narou_novels(keyword='', biggenre=None, genre=None, limit=20, start=1, order='new', is_r18=False,
                     min_length=None, max_length=None, kaiwaritu=None, sasie=None,
                     min_time=None, max_time=None, not_genres=None, not_biggenres=None,
                     istensei=None, istenni=None, iszankoku=None, is_stop=None, is_bl=None, is_gl=None, is_r15=None):
    """
    なろう小説APIを使って小説をキーワード検索する（R18対応・詳細検索オプション付き）

    Parameters:
    -----------
    keyword : str
        検索したいキーワード
    biggenre : int
        大ジャンル (1=恋愛, 2=ファンタジー, 3=文芸, 4=SF, 99=その他, 98=ノンジャンル)
    genre : int or str
        ジャンル (101=異世界〔恋愛〕, 102=現実世界〔恋愛〕, etc.)
    limit : int
        取得する小説の数 (最大500)
    start : int
        取得開始位置
    order : str
        並び順 (new=新着更新順, favnovelcnt=ブックマーク数順, etc.)
    is_r18 : bool
        R18作品を検索するかどうか
    min_length : int
        最小文字数
    max_length : int
        最大文字数
    kaiwaritu : str
        会話率 ("10-50"のように範囲指定可)
    sasie : str
        挿絵数 ("1-5"のように範囲指定可)
    min_time : int
        最小読了時間(分)
    max_time : int
        最大読了時間(分)
    not_genres : list or str
        除外するジャンル
    not_biggenres : list or str
        除外する大ジャンル
    istensei : int
        異世界転生モノ (1=含む, 0=指定なし)
    istenni : int
        異世界転移モノ (1=含む, 0=指定なし)
    iszankoku : int
        残酷な描写あり (1=含む, 0=指定なし)
    is_stop : int
        連載状況 (1=長期連載停止中除く, 2=長期連載停止中のみ, 0=指定なし)
    is_bl : int
        BL要素 (1=含む, 0=指定なし)
    is_gl : int
        GL要素 (1=含む, 0=指定なし)
    is_r15 : int
        R15要素 (1=含む, 0=指定なし)

    Returns:
    --------
    pd.DataFrame
        検索結果をDataFrameで返す
    """
    # URLエンコード
    encoded_keyword = urllib.parse.quote(keyword, encoding='utf-8')

    # APIのベースURL（R18かどうかで異なる）
    if is_r18:
        base_url = "https://api.syosetu.com/novel18api/api/"
        site_url = "https://novel18.syosetu.com/"
    else:
        base_url = "https://api.syosetu.com/novelapi/api/"
        site_url = "https://ncode.syosetu.com/"

    # パラメータ設定
    params = {
        "out": "json",
        "word": encoded_keyword,
        "lim": limit,
        "st": start,
        "order": order,
    }

    # 大ジャンル指定
    if biggenre is not None:
        params["biggenre"] = biggenre

    # ジャンル指定
    if genre is not None:
        params["genre"] = genre

    # 文字数指定
    if min_length is not None:
        params["minlen"] = min_length
    if max_length is not None:
        params["maxlen"] = max_length

    # 会話率
    if kaiwaritu is not None:
        params["kaiwaritu"] = kaiwaritu

    # 挿絵数
    if sasie is not None:
        params["sasie"] = sasie

    # 読了時間
    if min_time is not None:
        params["mintime"] = min_time
    if max_time is not None:
        params["maxtime"] = max_time

    # 除外ジャンル
    if not_genres is not None:
        params["notgenre"] = not_genres

    # 除外大ジャンル
    if not_biggenres is not None:
        params["notbiggenre"] = not_biggenres

    # 要素設定
    if istensei is not None and istensei == 1:
        params["istensei"] = 1
    if istenni is not None and istenni == 1:
        params["istenni"] = 1
    if iszankoku is not None and iszankoku == 1:
        params["iszankoku"] = 1
    if is_bl is not None and is_bl == 1:
        params["isbl"] = 1
    if is_gl is not None and is_gl == 1:
        params["isgl"] = 1
    if is_r15 is not None and is_r15 == 1:
        params["isr15"] = 1

    # 連載状況
    if is_stop is not None and is_stop > 0:
        params["stop"] = is_stop

    # URLパラメータを作成
    url_params = "&".join([f"{k}={v}" for k, v in params.items() if v])
    url = f"{base_url}?{url_params}"

    print(f"リクエストURL: {url}")

    try:
        # APIリクエスト
        response = requests.get(url)
        response.raise_for_status()

        # JSONデータを取得
        data = response.json()

        # 最初の要素は全作品数なので除外
        all_count = data[0]["allcount"]
        novels = data[1:]

        print(f"全作品数: {all_count}")

        if not novels:
            print("検索結果が見つかりませんでした。")
            return pd.DataFrame()

        # DataFrameに変換
        df = pd.DataFrame(novels)

        # 表示用に列を選択・整形
        if not df.empty:
            # 必要な列だけを選択
            columns_to_display = [
                "title", "writer", "story", "keyword",
                "genre", "ncode", "general_lastup", "length",
                "global_point", "fav_novel_cnt"
            ]

            # 存在する列だけを選択
            available_columns = [col for col in columns_to_display if col in df.columns]
            df_display = df[available_columns]

            # ジャンルコードを名前に変換
            genre_map = {
                101: "異世界〔恋愛〕",
                102: "現実世界〔恋愛〕",
                201: "ハイファンタジー〔ファンタジー〕",
                202: "ローファンタジー〔ファンタジー〕",
                301: "純文学〔文芸〕",
                302: "ヒューマンドラマ〔文芸〕",
                303: "歴史〔文芸〕",
                304: "推理〔文芸〕",
                305: "ホラー〔文芸〕",
                306: "アクション〔文芸〕",
                307: "コメディー〔文芸〕",
                401: "VRゲーム〔SF〕",
                402: "宇宙〔SF〕",
                403: "空想科学〔SF〕",
                404: "パニック〔SF〕",
                9901: "童話〔その他〕",
                9902: "詩〔その他〕",
                9903: "エッセイ〔その他〕",
                9904: "リプレイ〔その他〕",
                9999: "その他〔その他〕",
                9801: "ノンジャンル〔ノンジャンル〕"
            }

            if "genre" in df_display.columns:
                df_display["genre"] = df_display["genre"].map(lambda x: genre_map.get(x, str(x)))

            # Nコードにリンクを追加
            if "ncode" in df_display.columns:
                df_display["ncode"] = df_display["ncode"].map(
                    lambda x: f'<a href="{site_url}{x}/" target="_blank">{x}</a>'
                )

            # R18フラグを追加
            df_display["r18"] = "R18" if is_r18 else "一般"

            return df_display

        return df

    except Exception as e:
        print(f"エラーが発生しました: {e}")
        return pd.DataFrame()

# 結果の表示形式を整えるヘルパー関数
def display_with_clickable_links(df):
    """DataFrameをHTMLで表示し、リンクをクリック可能にする"""
    try:
        # DataFrameをスタイル付きHTMLに変換
        styled_df = df.style.set_properties(**{
            'text-align': 'left',
            'white-space': 'pre-wrap',
            'max-width': '500px',
            'overflow': 'hidden',
            'text-overflow': 'ellipsis'
        })

        # 文字数に応じて背景色を変える
        if 'length' in df.columns:
            styled_df = styled_df.background_gradient(cmap='Blues', subset=['length'])

        # ブックマーク数に応じて背景色を変える
        if 'fav_novel_cnt' in df.columns:
            styled_df = styled_df.background_gradient(cmap='Oranges', subset=['fav_novel_cnt'])

        # あらすじを短く切り詰める
        if 'story' in df.columns:
            df['story'] = df['story'].apply(lambda x: x[:150] + '...' if len(x) > 150 else x)

        # HTMLとして表示
        display(HTML(styled_df.to_html(escape=False)))
    except Exception as e:
        print(f"表示エラー: {e}")
        display(HTML(df.to_html(escape=False)))

# 検索結果をCSVにエクスポートする関数
def export_to_csv(df, filename='narou_search_result.csv'):
    """検索結果をCSVファイルにエクスポートする"""
    try:
        # エクスポート用のデータフレームを作成
        export_df = df.copy()

        # HTMLタグからNコードを抽出しつつ、リンクURLも別列に保存
        if 'ncode' in export_df.columns:
            # ncode列からコードを抽出
            export_df['ncode_id'] = export_df['ncode'].str.extract(r'>(.+)<')

            # リンクURLを抽出して新しい列に保存
            export_df['novel_url'] = export_df['ncode'].str.extract(r'href="([^"]+)"')

            # 元のHTMLリンクを削除
            export_df.drop('ncode', axis=1, inplace=True)

            # 列の順序を調整（ncode_idをわかりやすい位置に）
            cols = export_df.columns.tolist()
            # ncode_idを先頭近くに移動
            if 'title' in cols:
                title_pos = cols.index('title')
                cols.remove('ncode_id')
                cols.insert(title_pos + 1, 'ncode_id')
            # URLを末尾に
            if 'novel_url' in cols:
                cols.remove('novel_url')
                cols.append('novel_url')

            export_df = export_df[cols]

        # UTF-8でCSVとして保存
        export_df.to_csv(filename, index=False, encoding='utf-8-sig')
        print(f"{filename} にエクスポートしました。")
        print(f"小説へのリンクは 'novel_url' 列に保存されています。")

        # Google Colabの場合、ダウンロードリンクを表示
        try:
            from google.colab import files
            files.download(filename)
            print("ダウンロードを開始しました。")
        except:
            print("ファイルをダウンロードするには、ノートブックのファイルブラウザからアクセスしてください。")

    except Exception as e:
        print(f"エクスポートエラー: {e}")
        import traceback
        traceback.print_exc()

# Google Colabで対話的に使用するためのUIを追加
def create_search_ui():
    """Google Colab用のタブ付き検索UIを作成"""
    from ipywidgets import widgets
    from IPython.display import display, clear_output

    # タブの作成
    tab = widgets.Tab()

    # タブ1: 基本検索
    basic_tab = widgets.VBox()

    # 基本検索の入力フォーム
    keyword_input = widgets.Text(description='キーワード:', placeholder='検索キーワードを入力')

    # R18選択
    r18_select = widgets.Checkbox(description='R18作品を検索', value=False)

    # ジャンル選択
    genre_select = widgets.Dropdown(
        options=[
            ('指定なし', None),
            ('恋愛', 1),
            ('ファンタジー', 2),
            ('文芸', 3),
            ('SF', 4),
            ('その他', 99),
            ('ノンジャンル', 98)
        ],
        value=1,
        description='ジャンル:'
    )

    # 並び順選択
    order_select = widgets.Dropdown(
        options=[
            ('新着更新順', 'new'),
            ('ブックマーク数順', 'favnovelcnt'),
            ('レビュー数順', 'reviewcnt'),
            ('総合ポイント順', 'hyoka'),
            ('日間ポイント順', 'dailypoint'),
            ('週間ポイント順', 'weeklypoint'),
            ('月間ポイント順', 'monthlypoint'),
            ('文字数の多い順', 'lengthdesc'),
            ('文字数の少ない順', 'lengthasc')
        ],
        value='new',
        description='並び順:'
    )

    # 取得件数
    limit_slider = widgets.IntSlider(
        value=20,
        min=1,
        max=100,
        step=5,
        description='件数:',
        continuous_update=False
    )

    # 基本検索タブに追加
    basic_tab.children = [
        keyword_input,
        widgets.HBox([r18_select, genre_select]),
        widgets.HBox([order_select, limit_slider])
    ]

    # タブ2: 詳細検索
    advanced_tab = widgets.VBox()

    # 詳細なジャンル選択
    detailed_genre_select = widgets.Dropdown(
        options=[
            ('指定なし', None),
            ('異世界〔恋愛〕', 101),
            ('現実世界〔恋愛〕', 102),
            ('ハイファンタジー〔ファンタジー〕', 201),
            ('ローファンタジー〔ファンタジー〕', 202),
            ('純文学〔文芸〕', 301),
            ('ヒューマンドラマ〔文芸〕', 302),
            ('歴史〔文芸〕', 303),
            ('推理〔文芸〕', 304),
            ('ホラー〔文芸〕', 305),
            ('アクション〔文芸〕', 306),
            ('コメディー〔文芸〕', 307),
            ('VRゲーム〔SF〕', 401),
            ('宇宙〔SF〕', 402),
            ('空想科学〔SF〕', 403),
            ('パニック〔SF〕', 404),
            ('童話〔その他〕', 9901),
            ('詩〔その他〕', 9902),
            ('エッセイ〔その他〕', 9903),
            ('リプレイ〔その他〕', 9904),
            ('その他〔その他〕', 9999),
            ('ノンジャンル〔ノンジャンル〕', 9801)
        ],
        value=None,
        description='詳細ジャンル:'
    )

    # 文字数範囲
    min_length = widgets.IntText(
        value=None,
        description='最小文字数:',
        placeholder='例: 10000'
    )

    max_length = widgets.IntText(
        value=None,
        description='最大文字数:',
        placeholder='例: 50000'
    )

    # 会話率範囲
    kaiwaritu = widgets.Text(
        value='',
        description='会話率:',
        placeholder='例: 30-70'
    )

    # 挿絵数範囲
    sasie = widgets.Text(
        value='',
        description='挿絵数:',
        placeholder='例: 0-5'
    )

    # 読了時間範囲
    min_time = widgets.IntText(
        value=None,
        description='最小読了時間(分):',
        placeholder='例: 10'
    )

    max_time = widgets.IntText(
        value=None,
        description='最大読了時間(分):',
        placeholder='例: 60'
    )

    # 作品要素
    istensei = widgets.Checkbox(description='異世界転生', value=False)
    istenni = widgets.Checkbox(description='異世界転移', value=False)
    iszankoku = widgets.Checkbox(description='残酷な描写あり', value=False)
    is_bl = widgets.Checkbox(description='BL', value=False)
    is_gl = widgets.Checkbox(description='GL', value=False)
    is_r15 = widgets.Checkbox(description='R15', value=False)

    # 連載状況
    is_stop = widgets.Dropdown(
        options=[
            ('指定なし', None),
            ('長期連載停止中を除く', 1),
            ('長期連載停止中のみ', 2)
        ],
        value=None,
        description='連載状況:'
    )

    # 詳細検索タブに追加
    advanced_tab.children = [
        detailed_genre_select,
        widgets.HBox([min_length, max_length]),
        widgets.HBox([kaiwaritu, sasie]),
        widgets.HBox([min_time, max_time]),
        widgets.HBox([istensei, istenni, iszankoku]),
        widgets.HBox([is_bl, is_gl, is_r15, is_stop])
    ]

    # タブをまとめる
    tab.children = [basic_tab, advanced_tab]
    tab.set_title(0, '基本検索')
    tab.set_title(1, '詳細検索')

    # 結果格納用の変数
    results_df = [None]  # リストで包むことで参照を保持

    # 実行ボタン
    search_button = widgets.Button(
        description='検索',
        button_style='primary'
    )

    # CSVエクスポートボタン
    export_button = widgets.Button(
        description='CSVエクスポート',
        button_style='info',
        disabled=True
    )

    # 結果表示領域
    output = widgets.Output()

    # 全体レイアウト
    layout = widgets.VBox([
        tab,
        widgets.HBox([search_button, export_button]),
        output
    ])

    # ボタンクリック時の処理
    def on_button_clicked(b):
        with output:
            clear_output()

            # 基本検索パラメータを取得
            search_keyword = keyword_input.value
            search_r18 = r18_select.value
            search_biggenre = genre_select.value
            search_order = order_select.value
            search_limit = limit_slider.value

            # 詳細検索パラメータを取得
            search_genre = detailed_genre_select.value

            # 文字数範囲
            search_min_length = min_length.value if min_length.value else None
            search_max_length = max_length.value if max_length.value else None

            # 会話率
            search_kaiwaritu = kaiwaritu.value if kaiwaritu.value else None

            # 挿絵数
            search_sasie = sasie.value if sasie.value else None

            # 読了時間
            search_min_time = min_time.value if min_time.value else None
            search_max_time = max_time.value if max_time.value else None

            # 作品要素
            search_istensei = 1 if istensei.value else None
            search_istenni = 1 if istenni.value else None
            search_iszankoku = 1 if iszankoku.value else None
            search_is_bl = 1 if is_bl.value else None
            search_is_gl = 1 if is_gl.value else None
            search_is_r15 = 1 if is_r15.value else None

            # 連載状況
            search_is_stop = is_stop.value

            # R18確認
            if search_r18:
                print("注意: R18作品の検索を実行します。18歳以上であることを確認してください。")

            print(f"検索条件: キーワード='{search_keyword}', R18={search_r18}")
            print("検索中...")

            # 検索実行
            df = search_narou_novels(
                keyword=search_keyword,
                biggenre=search_biggenre,
                genre=search_genre,
                limit=search_limit,
                order=search_order,
                is_r18=search_r18,
                min_length=search_min_length,
                max_length=search_max_length,
                kaiwaritu=search_kaiwaritu,
                sasie=search_sasie,
                min_time=search_min_time,
                max_time=search_max_time,
                istensei=search_istensei,
                istenni=search_istenni,
                iszankoku=search_iszankoku,
                is_bl=search_is_bl,
                is_gl=search_is_gl,
                is_r15=search_is_r15,
                is_stop=search_is_stop
            )

            # 結果を保存
            results_df[0] = df

            # エクスポートボタンの状態を設定
            export_button.disabled = df.empty

            if not df.empty:
                print(f"\n検索結果: {len(df)}件")
                display_with_clickable_links(df)
            else:
                print("検索結果がありませんでした。")

    # エクスポートボタンクリック時の処理
    def on_export_clicked(b):
        if results_df[0] is not None and not results_df[0].empty:
            with output:
                # ファイル名の作成
                filename = f"narou_search_{keyword_input.value}_{time.strftime('%Y%m%d_%H%M%S')}.csv"
                export_to_csv(results_df[0], filename)

    search_button.on_click(on_button_clicked)
    export_button.on_click(on_export_clicked)

    # UIを表示
    display(layout)

    return layout

def add_usage_instructions():
    """使い方の説明を表示する"""
    from IPython.display import display, HTML, Markdown

    instructions = """
    # なろう小説検索ツールの使い方

    ## 基本操作

    1. **基本検索タブ**で簡単な検索条件を設定:
       - キーワード: 検索したい単語を入力（複数の単語はスペースで区切ると AND 検索）
       - R18作品: チェックを入れるとR18作品を検索対象に
       - ジャンル: 大ジャンル（恋愛、ファンタジーなど）を選択
       - 並び順: 並べる順序（新着、ブックマーク数など）を選択
       - 件数: 取得する結果の件数を設定

    2. **詳細検索タブ**で詳細な条件を追加:
       - 詳細ジャンル: より細かいジャンル（異世界恋愛、現実世界恋愛など）
       - 文字数: 最小文字数と最大文字数で範囲指定
       - 会話率: 小説内の会話の割合を指定（例: 30-70）
       - 挿絵数: 挿絵の数を指定（例: 1-5、1-）
       - 読了時間: 読むのにかかる時間（分）を範囲指定
       - 作品要素: 含みたい要素（異世界転生、残酷描写など）にチェック
       - 連載状況: 長期連載停止中の作品を除外するかどうか

    3. **検索ボタン**をクリックして検索を実行

    4. 検索結果が表示されたら、**CSVエクスポート**ボタンでデータを保存可能

    ## 検索結果の見方

    - 各作品のNコードをクリックすると「小説家になろう」のサイトで作品を読めます
    - 文字数：背景色が濃いほど文字数が多い作品
    - ブックマーク数：背景色が濃いほどブックマークが多い人気作品

    ## 検索のコツ

    - 複数キーワードはスペースで区切る（AND検索）
    - 「異世界転生」と「異世界転移」の両方を探す場合は両方にチェック
    - 文字数とジャンルを組み合わせると良い作品が見つかりやすい
    - 「長期連載停止中を除く」にすると放置されていない作品だけ表示
    - R18作品を検索するには18歳以上である必要があります
    """

    display(Markdown(instructions))

# メイン関数
def main():
    """メイン実行関数"""
    print("なろう小説API検索ツール（R18対応・詳細検索タブ付き）")
    print("="*60)
    print("このツールは「小説家になろう」の作品をAPIを使って検索するものです。")
    print("基本タブと詳細タブの2つのタブで複数条件での検索が可能です。")
    print("-"*60)

    # 使い方を表示
    add_usage_instructions()

    # UIを作成
    create_search_ui()

# 使用例
if __name__ == "__main__":
    main()

# 以下はサンプルコードです。実行したい場合はコメントを外してください
"""
# 基本的な検索
df = search_narou_novels("学園", biggenre=1, limit=10, is_r18=False)
display_with_clickable_links(df)

# 詳細な条件指定
df = search_narou_novels(
    keyword="恋愛",
    biggenre=1,  # 恋愛
    genre=102,   # 現実世界恋愛
    limit=20,
    order="favnovelcnt",  # ブックマーク数順
    is_r18=False,
    min_length=30000,
    max_length=100000,
    kaiwaritu="30-70",  # 会話率30%〜70%
    sasie="0-3",        # 挿絵0〜3枚
    istensei=None,      # 異世界転生は指定なし
    is_stop=1           # 長期連載停止中を除く
)
display_with_clickable_links(df)
"""

なろう小説API検索ツール（R18対応・詳細検索タブ付き）
このツールは「小説家になろう」の作品をAPIを使って検索するものです。
基本タブと詳細タブの2つのタブで複数条件での検索が可能です。
------------------------------------------------------------



    # なろう小説検索ツールの使い方
    
    ## 基本操作
    
    1. **基本検索タブ**で簡単な検索条件を設定:
       - キーワード: 検索したい単語を入力（複数の単語はスペースで区切ると AND 検索）
       - R18作品: チェックを入れるとR18作品を検索対象に
       - ジャンル: 大ジャンル（恋愛、ファンタジーなど）を選択
       - 並び順: 並べる順序（新着、ブックマーク数など）を選択
       - 件数: 取得する結果の件数を設定
    
    2. **詳細検索タブ**で詳細な条件を追加:
       - 詳細ジャンル: より細かいジャンル（異世界恋愛、現実世界恋愛など）
       - 文字数: 最小文字数と最大文字数で範囲指定
       - 会話率: 小説内の会話の割合を指定（例: 30-70）
       - 挿絵数: 挿絵の数を指定（例: 1-5、1-）
       - 読了時間: 読むのにかかる時間（分）を範囲指定
       - 作品要素: 含みたい要素（異世界転生、残酷描写など）にチェック
       - 連載状況: 長期連載停止中の作品を除外するかどうか
    
    3. **検索ボタン**をクリックして検索を実行
    
    4. 検索結果が表示されたら、**CSVエクスポート**ボタンでデータを保存可能
    
    ## 検索結果の見方
    
    - 各作品のNコードをクリックすると「小説家になろう」のサイトで作品を読めます
    - 文字数：背景色が濃いほど文字数が多い作品
    - ブックマーク数：背景色が濃いほどブックマークが多い人気作品
    
    ## 検索のコツ
    
    - 複数キーワードはスペースで区切る（AND検索）
    - 「異世界転生」と「異世界転移」の両方を探す場合は両方にチェック
    - 文字数とジャンルを組み合わせると良い作品が見つかりやすい
    - 「長期連載停止中を除く」にすると放置されていない作品だけ表示
    - R18作品を検索するには18歳以上である必要があります
    

'\n# 基本的な検索\ndf = search_narou_novels("学園", biggenre=1, limit=10, is_r18=False)\ndisplay_with_clickable_links(df)\n\n# 詳細な条件指定\ndf = search_narou_novels(\n    keyword="恋愛", \n    biggenre=1,  # 恋愛\n    genre=102,   # 現実世界恋愛\n    limit=20, \n    order="favnovelcnt",  # ブックマーク数順\n    is_r18=False,\n    min_length=30000,\n    max_length=100000,\n    kaiwaritu="30-70",  # 会話率30%〜70%\n    sasie="0-3",        # 挿絵0〜3枚\n    istensei=None,      # 異世界転生は指定なし\n    is_stop=1           # 長期連載停止中を除く\n)\ndisplay_with_clickable_links(df)\n'